# Analysis of Features by Classification Accuracy

In [1]:
import init
import common.constants as cn
from common.trinary_data import TrinaryData
import classifier.main_multi_classifier_feature_optimizer as main
from common.data_provider import DataProvider
from common_python.plots import util_plots
from plots import util_plots as xutil_plots

import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

%matplotlib inline

In [2]:
fit_result_dct = main.getFitResultFromPersister()

In [3]:
for cl in fit_result_dct.keys():
    length_stg = str([len(fr.sels) for fr in fit_result_dct[cl]])
    score_stg = str([fr.sels_score for fr in fit_result_dct[cl]])
    print("%d: %s\n   %s" % (cl, length_stg, score_stg))

0: [1, 1, 1, 2, 2, 3, 2, 3, 2, 2, 4, 3, 2, 5, 3]
   [1.0, 1.0, 0.85, 1.0, 1.0, 0.79, 1.0, 0.88, 1.0, 0.85, 0.99, 1.0, 0.87, 0.82, 1.0]
1: [1, 1, 2, 2, 5, 5, 2, 4, 2, 2, 2, 2, 5, 1, 6]
   [0.945, 0.98, 0.96, 0.94, 0.965, 0.97, 0.955, 0.995, 0.99, 0.955, 0.925, 0.96, 0.965, 0.945, 0.935]
2: [1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1]
   [0.935, 0.855, 0.855, 0.815, 0.87, 0.855, 0.86, 0.865, 0.885, 0.86, 0.84, 0.865, 0.885, 0.865, 0.835]
3: [2, 3, 3, 2, 2, 3, 2, 5, 2, 2, 1, 2, 2, 2, 4]
   [1.0, 0.93, 0.98, 0.985, 0.98, 0.975, 0.97, 0.89, 0.92, 0.945, 0.945, 0.905, 0.965, 0.91, 0.98]
4: [1, 1, 1]
   [0.935, 0.795, 0.91]
